In [1]:
from flask import Flask, render_template, redirect
from splinter import Browser
import os
from bs4 import BeautifulSoup as bs
import requests
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import pymongo
import time

# Part 1: Scraping
### NASA Mars News

In [2]:
# Setup splinter
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)



====== WebDriver manager ======
Current google-chrome version is 102.0.5005
Get LATEST chromedriver version for 102.0.5005 google-chrome
Trying to download new driver from https://chromedriver.storage.googleapis.com/102.0.5005.61/chromedriver_win32.zip
Driver has been saved in cache [C:\Users\ninap\.wdm\drivers\chromedriver\win32\102.0.5005.61]


In [3]:
# Use browser to visit the URL
url = 'https://redplanetscience.com/'
browser.visit(url)

In [4]:
# Wait 5 seconds
time.sleep(5)

In [5]:
# Retrieve page
html = browser.html

In [6]:
# Create Beautiful Soup Object
soup = bs(html, 'html.parser')
type(soup)

bs4.BeautifulSoup

In [7]:
# Extract the latest news title
latest_news_title = soup.find('div', class_='content_title').get_text()
print(latest_news_title)

While Stargazing on Mars, NASA's Curiosity Rover Spots Earth and Venus


In [8]:
# Extract paragraph text of the latest news story
latest_news_paragraph = soup.find('div', class_='article_teaser_body').get_text()
print(latest_news_paragraph)

This new portrait of the Red Planet's neighbors was taken during a time when there's more dust in the air on Mars.


### JPL Mars Space Images-Featured Image

In [9]:
# Setup splinter
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)



====== WebDriver manager ======
Current google-chrome version is 102.0.5005
Get LATEST chromedriver version for 102.0.5005 google-chrome
Driver [C:\Users\ninap\.wdm\drivers\chromedriver\win32\102.0.5005.61\chromedriver.exe] found in cache


In [10]:
# Use browser to visit the URL
images_url = 'https://spaceimages-mars.com/'
browser.visit(images_url)

In [11]:
# Retrieve page
html = browser.html

In [12]:
# URL of the Featured Mars Image
featured_image_url = 'https://spaceimages-mars.com/image/featured/mars3.jpg'
browser.visit(featured_image_url)

### Mars Facts

In [13]:
def mars_facts():
    facts_url = 'https://galaxyfacts-mars.com/'
    # Read html table
    tables = pd.read_html(facts_url)
    df=tables[0]
    df.columns=['Description', 'Mars', 'Earth']
    df.set_index('Description', inplace=True)
    df=df.iloc[1:]
    # Clean table by removing unwanted newlines 
    return df.to_html(classes='table table-striped').replace('\n', '')

In [14]:
# Call the function
mars_facts()

'<table border="1" class="dataframe table table-striped">  <thead>    <tr style="text-align: right;">      <th></th>      <th>Mars</th>      <th>Earth</th>    </tr>    <tr>      <th>Description</th>      <th></th>      <th></th>    </tr>  </thead>  <tbody>    <tr>      <th>Diameter:</th>      <td>6,779 km</td>      <td>12,742 km</td>    </tr>    <tr>      <th>Mass:</th>      <td>6.39 × 10^23 kg</td>      <td>5.97 × 10^24 kg</td>    </tr>    <tr>      <th>Moons:</th>      <td>2</td>      <td>1</td>    </tr>    <tr>      <th>Distance from Sun:</th>      <td>227,943,824 km</td>      <td>149,598,262 km</td>    </tr>    <tr>      <th>Length of Year:</th>      <td>687 Earth days</td>      <td>365.24 days</td>    </tr>    <tr>      <th>Temperature:</th>      <td>-87 to -5 °C</td>      <td>-88 to 58°C</td>    </tr>  </tbody></table>'

### Mars Hemispheres

In [36]:
# Setup splinter
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)



====== WebDriver manager ======
Current google-chrome version is 102.0.5005
Get LATEST chromedriver version for 102.0.5005 google-chrome
Trying to download new driver from https://chromedriver.storage.googleapis.com/102.0.5005.61/chromedriver_win32.zip
Driver has been saved in cache [C:\Users\ninap\.wdm\drivers\chromedriver\win32\102.0.5005.61]


In [37]:
# Use browser to visit the URL
all_hemispheres_url = 'https://marshemispheres.com/'
browser.visit(all_hemispheres_url)

In [38]:
# Create Beautiful Soup Object
soup = bs(browser.html, 'html.parser')
type(soup)

bs4.BeautifulSoup

In [39]:
# Create dictionary to store the full resolution hemisphere images and hemisphere titles
hemisphere_image_list = []

In [41]:
# Select first hemisphere link
#browser.find_by_css('img.thumb')[0].click()

# Download first hemisphere link
#img_url = browser.find_by_text('Sample')['href']
#img_url
#browser.back()

#store_img_urls = []

for i in range(3):
    browser.find_by_css('img.thumb')[i].click() 
    browser.find_by_text('Sample')['href']
#    browser.find_by_text('Sample').first()['href']
    browser.back()
    print()

hemisphere_image_list["img_url"] = img_url
#hemisphere_image_list.append({"img url": image_url})

NameError: name 'hemisphere_data' is not defined

In [26]:
hemisphere_image_list

{'img_url': 'https://marshemispheres.com/images/full.jpg'}

In [ ]:
# Create dictionary to store the full resolution hemisphere images and hemisphere titles

hemisphere_image_dict = {}

# Loop over item elements
items = soup.find_all('item')

for item in items:
    all_hemispheres_url = item.find("a")["href"]
    browser.visit(ea_hemisphere_url)
    
    # Parse each hemisphere webpage
    hemisphere_item_html = browser.html
    hemisphere_soup = soup(hemisphere_item_html, "html.parser")
    
    title = hemisphere_soup().find("alt", "class_" = "title").text
    
    full_size_image_download = hemisphere_soup.find("div", "class_" = "downloads")
    
    image_url = downloads.find("a")["href"]
    
    hemisphere_image_dict.append({"title": title, "img url": image_url})

In [ ]:
# Print the dictionary of image URLs and their titles
hemisphere_image_dict